# This notebook computes the stacks for all pairs from single sources

In [1]:
import os,random,time,pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from obspy.signal.filter import bandpass
from seisgo import noise,utils,plotting
from seisgo.stacking import seisstack as stack
import pygmt as gmt

## Global parameters

In [3]:
dataroot='data_stacking'
comp="ZZ"
raw=os.path.join(dataroot,"MERGED_cascadia_raw")
tnorm=os.path.join(dataroot,"MERGED_cascadia_tnorm")
raw_xz=os.path.join(dataroot,"MERGED_XZ_raw")
tnorm_xz=os.path.join(dataroot,"MERGED_XZ_tnorm")

#
figformat="png"
figdpi=200

dlabels=["Raw","One-bit"]
cmap="seismic"
methods=["linear","robust","selective","cluster","pws","tfpws","nroot","acf"]
method_labels=["Linear","Robust","Selective","Cluster","PWS","tf-PWS","$N^{th}$-root","ACF"]
colors=["k","b","c","y","g","orange","r","m"]

## Cascadia amphibious array all pairs from the same virtual source

In [ ]:
source="7D.J33A"
network="cascadia"
#manually created file list including all "source" station (either as a source or as a receiver)
dfilelist_raw=raw+"/all_"+source+".txt"
dfilelist_tnorm=tnorm+"/all_"+source+".txt"

In [4]:
flist_all=[dfilelist_raw,dfilelist_tnorm]
stack_all=dict()
dist_all=dict()
t0=time.time()
for i,fn in enumerate(flist_all):
    fh=open(fn,'r')
    flist=fh.readlines()
    #get general parameters
    if i==0:
        cdict_temp=noise.extract_corrdata(raw+"/"+flist[0].replace("\n",""),comp=comp)
        cdata0=cdict_temp[list(cdict_temp.keys())[0]][comp]
        npts=cdata0.data.shape[1] #get number of points.
        maxlag=cdata0.lag
        dt=cdata0.dt
    print(dlabels[i])
    stack_all[dlabels[i]]=np.ndarray((len(methods),len(flist),npts)) #3 dimensional array
    if i==0:
        stack_all["maxlag"]=maxlag
        stack_all["dt"]=dt
    dist_all[dlabels[i]]=np.ndarray((len(flist)))
    for j,f0 in enumerate(flist):
        f=f0.replace("\n","")
        print(f+"  --> "+str(j))
        if i==0:
            cdict=noise.extract_corrdata(raw+"/"+f,comp=comp)
        else:
            cdict=noise.extract_corrdata(tnorm+"/"+f,comp=comp)
        p=list(cdict.keys())[0]
        cdata=cdict[p][comp]
        netsta1 = cdata.net[0]+'.'+cdata.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
        netsta2 = cdata.net[1]+'.'+cdata.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]
    
        dist_all[dlabels[i]][j]=cdata.dist
        for k,m in enumerate(methods):
            ds0 = stack(cdata.data.copy(),m)            
            ds=ds0.copy()
            if netsta2.lower() == source.lower(): ds=np.flip(ds)
            stack_all[dlabels[i]][k,j,:]=ds

with open('xcorr_stacks_allpair_'+network+'.pk','wb') as xf:
    pickle.dump(stack_all,xf)
    pickle.dump(dist_all,xf)
    
print("finished in %f s"%(time.time() - t0))

Raw
7D.FN01A/7D.FN01A_7D.J33A.h5  --> 0
7D.FN07A/7D.FN07A_7D.J33A.h5  --> 1
7D.FN08A/7D.FN08A_7D.J33A.h5  --> 2
7D.FN12A/7D.FN12A_7D.J33A.h5  --> 3
7D.FN14A/7D.FN14A_7D.J33A.h5  --> 4
7D.FN16A/7D.FN16A_7D.J33A.h5  --> 5
7D.FN18A/7D.FN18A_7D.J33A.h5  --> 6
7D.G03A/7D.G03A_7D.J33A.h5  --> 7
7D.G30A/7D.G30A_7D.J33A.h5  --> 8
7D.J06A/7D.J06A_7D.J33A.h5  --> 9
7D.J23A/7D.J23A_7D.J33A.h5  --> 10
7D.J25A/7D.J25A_7D.J33A.h5  --> 11
7D.J25B/7D.J25B_7D.J33A.h5  --> 12
7D.J26A/7D.J26A_7D.J33A.h5  --> 13
7D.J28A/7D.J28A_7D.J33A.h5  --> 14
7D.J29A/7D.J29A_7D.J33A.h5  --> 15
7D.J30A/7D.J30A_7D.J33A.h5  --> 16
7D.J31A/7D.J31A_7D.J33A.h5  --> 17
7D.J33A/7D.J33A_7D.J33B.h5  --> 18
7D.J33A/7D.J33A_7D.J34A.h5  --> 19
7D.J33A/7D.J33A_7D.J35A.h5  --> 20
7D.J33A/7D.J33A_7D.J36A.h5  --> 21
7D.J33A/7D.J33A_7D.J37A.h5  --> 22
7D.J33A/7D.J33A_7D.J38A.h5  --> 23
7D.J33A/7D.J33A_7D.J39A.h5  --> 24
7D.J33A/7D.J33A_7D.J41A.h5  --> 25
7D.J33A/7D.J33A_7D.J42A.h5  --> 26
7D.J33A/7D.J33A_7D.J43A.h5  --> 27
7D.J33A/7D.J

In [52]:
receiver_name_all=dict()
receiver_ele_all=dict()

for i,fn in enumerate(flist_all):
    fh=open(fn,'r')
    flist=fh.readlines()
    #get general parameters
    if i==0:
        cdict_temp=noise.extract_corrdata(raw+"/"+flist[0].replace("\n",""),comp=comp)
        cdata0=cdict_temp[list(cdict_temp.keys())[0]][comp]
        npts=cdata0.data.shape[1] #get number of points.
        maxlag=cdata0.lag
        dt=cdata0.dt
    print(dlabels[i])
    receiver_ele_all[dlabels[i]]=np.ndarray((len(flist)))
    receiver_name_all[dlabels[i]]=[]
    for j,f0 in enumerate(flist):
        f=f0.replace("\n","")
        print(f+"  --> "+str(j))
        if i==0:
            cdict=noise.extract_corrdata(raw+"/"+f,comp=comp)
        else:
            cdict=noise.extract_corrdata(tnorm+"/"+f,comp=comp)
        p=list(cdict.keys())[0]
        cdata=cdict[p][comp]
        netsta1 = cdata.net[0]+'.'+cdata.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
        netsta2 = cdata.net[1]+'.'+cdata.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]
        if netsta2.lower() == source.lower():
            receiver_ele_all[dlabels[i]][j]=cdata.ele[0]
            receiver_name_all[dlabels[i]].append(netsta1)
        else:
            receiver_ele_all[dlabels[i]][j]=cdata.ele[1]
            receiver_name_all[dlabels[i]].append(netsta2)

with open('xcorr_stacks_allpair_receiverinfo_'+network+'.pk','wb') as xf:
    pickle.dump(receiver_ele_all,xf)
    pickle.dump(receiver_name_all,xf)

### XZ pairs from a single virtual source

In [6]:
XZ_source="XZ.A02"
network="XZ"
flist_xz=[utils.get_filelist(raw_xz+"/"+XZ_source,"h5"),
          utils.get_filelist(tnorm_xz+"/"+XZ_source,"h5")]
stack_all_xz=dict()
dist_all_xz=dict()
t1=time.time()
for i,flist in enumerate(flist_xz):
    #get general parameters
    if i==0:
        cdict_temp=noise.extract_corrdata(flist[0].replace("\n",""),comp=comp)
        cdata0=cdict_temp[list(cdict_temp.keys())[0]][comp]
        npts_xz=cdata0.data.shape[1] #get number of points.
        maxlag_xz=cdata0.lag
        dt_xz=cdata0.dt
    print(dlabels[i])
    stack_all_xz[dlabels[i]]=np.ndarray((len(methods),len(flist),npts_xz)) #3 dimensional array
    if i==0:
        stack_all_xz["maxlag"]=maxlag
        stack_all_xz["dt"]=dt
    dist_all_xz[dlabels[i]]=np.ndarray((len(flist)))
    for j,f0 in enumerate(flist):
        f=f0.replace("\n","")
        print(f+"  --> "+str(j))
        cdict=noise.extract_corrdata(f,comp=comp)
        p=list(cdict.keys())[0]
        cdata=cdict[p][comp]
        netsta1 = cdata.net[0]+'.'+cdata.sta[0]#+'.'+c.loc[0]+'.'+c.chan[0]
        netsta2 = cdata.net[1]+'.'+cdata.sta[1]#+'.'+c.loc[1]+'.'+c.chan[1]
    
        dist_all_xz[dlabels[i]][j]=cdata.dist
        for k,m in enumerate(methods):
            ds0 = stack(cdata.data.copy(),m)
            ds=ds0.copy()
            if netsta2.lower() == XZ_source.lower(): ds=np.flip(ds)
            stack_all_xz[dlabels[i]][k,j,:]=ds

with open('xcorr_stacks_allpair_'+network+'.pk','wb') as xf:
    pickle.dump(stack_all_xz,xf)
    pickle.dump(dist_all_xz,xf)
    
print("finished in %f s"%(time.time() - t1))

Raw
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A03.h5  --> 0
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A04.h5  --> 1
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A05.h5  --> 2
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A06.h5  --> 3
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A07.h5  --> 4
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A08.h5  --> 5
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A09.h5  --> 6
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A10.h5  --> 7
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A11.h5  --> 8
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A12.h5  --> 9
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A13.h5  --> 10
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A14.h5  --> 11
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A15.h5  --> 12
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A16.h5  --> 13
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A17.h5  --> 14
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A18.h5  --> 15
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A

/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/seisgo/stacking.py:147: RuntimeWarning: invalid value encountered in true_divide
  p = np.power((S1-S2)/(S2*(N-1)),g)


data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A30.h5  --> 29
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A31.h5  --> 30
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A32.h5  --> 31
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A33.h5  --> 32
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A34.h5  --> 33
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A35.h5  --> 34
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A36.h5  --> 35
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A37.h5  --> 36
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A38.h5  --> 37
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A39.h5  --> 38
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A40.h5  --> 39
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A41.h5  --> 40
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A42.h5  --> 41
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A43.h5  --> 42
data_stacking/MERGED_XZ_raw/XZ.A02/XZ.A02_XZ.A44.h5  --> 43
One-bit
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A03.h5  --> 0
data_stacking/MERGED_XZ_tnorm/X

/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/opt/anaconda3/envs/seisgo/lib/python3.8/site-packages/seisgo/stacking.py:147: RuntimeWarning: invalid value encountered in true_divide
  p = np.power((S1-S2)/(S2*(N-1)),g)


data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A30.h5  --> 29
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A31.h5  --> 30
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A32.h5  --> 31
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A33.h5  --> 32
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A34.h5  --> 33
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A35.h5  --> 34
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A36.h5  --> 35
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A37.h5  --> 36
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A38.h5  --> 37
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A39.h5  --> 38
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A40.h5  --> 39
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A41.h5  --> 40
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A42.h5  --> 41
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A43.h5  --> 42
data_stacking/MERGED_XZ_tnorm/XZ.A02/XZ.A02_XZ.A44.h5  --> 43
finished in 16099.584147 s
